In [13]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

train_sentences = ['an apple', 'an orange']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent)))
                  for sent in train_sentences]
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['an apple', 'an ant']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent)))
                  for sent in test_sentences]

test_data, _ = padded_everygram_pipeline(n, tokenized_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_text)

for i, test in enumerate(test_data):
    print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))


MLE Estimates: [(('an', ()), 0.5), (('apple', ()), 0.25)]
MLE Estimates: [(('an', ()), 0.5), (('ant', ()), 0.0)]
PP(an apple):2.8284271247461903
PP(an ant):inf


In [17]:
n = 2
a, b = padded_everygram_pipeline(n, tokenized_text)
print(list(list(x) for x in a), list(b))

[[('<s>',), ('<s>', 'an'), ('an',), ('an', 'apple'), ('apple',), ('apple', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'an'), ('an',), ('an', 'ant'), ('ant',), ('ant', '</s>'), ('</s>',)]] ['<s>', 'an', 'apple', '</s>', '<s>', 'an', 'ant', '</s>']


# Исследование n-грамм модели текста

1. Создайте корпус текстов из нескольких литературных произведений, выберите хотя бы два больших текста
ваших любимых писателей.
2. Далее вам надо разбить текст на предложения (см. ранее `sent_tokenze`), соответственно, вы получаете
   большой список предложений, которые мы будем использовать для исследований.
3. Разбейте список предложений случайным образом на три части: для обучения (train set), для настройки
 (validation set), для тестирования качества (test set). Для этого перемешайте список предложений, возьмите
  первые 80% как train set, следующие 10% как validation set, оставшиеся 10% как test set.

  Пример кода для перемешивания:

In [18]:
from random import shuffle, seed

# инициализация генератора случайных чисел, случайные числа все время будут
# одинаковые, это удобно для отладки, после отладки эту строку надо убрать.
seed(42)

l = ["мы", "слова", "которые", "надо", "перемешать"]
shuffle(l)
print(l)

['надо', 'слова', 'которые', 'перемешать', 'мы']


4. Предложения из всех трёх множеств нужно обработать. Для обучающего множества можно пользоваться
  встроенной в nltk функцией `padded_everygram_pipeline`.

  Эта функция делает сразу много действий, она дополняет предложения техническими словами `<s>` и `</s>`
  для начала и конца, возвращает `n_grams`, который для каждого предложения содержит список всех n-грамм
  этого предложения, причем, если мы указываем n=3 в качестве аргумента, будут построены и 1-, и 2-,
  и 3-граммы. В `words` возвращается список всех слов, чтобы построить словарь на их основе:

In [30]:
sentences = [
    ["моё", "первое", "предложение", "!"],
    ["моё", "второе", "предложение", "."],
    ["еще", "одно", "предложение", "."]
]

n = 3
n_grams, words = padded_everygram_pipeline(n, sentences)

print("все слова:")
print(list(words)) # нужно делать list(), потому что words это генератор
print("все n-граммы")
for sentence_n_grams in n_grams:
    print(list(sentence_n_grams))  # нужно делать list(), потому что sentence_n_grams это генератор

# небольшое замечание, padded_everygram_pipeline возвращает генераторы
# их можно использовать только один раз. Например, их можно распечатать,
# или можно обучить через них модель. После этого их надо
# создавать заново. Вы можете столкнуться с этим при отладке,
# модель не обучается после того, как вы всё распечатали.

все слова:
['<s>', '<s>', 'моё', 'первое', 'предложение', '!', '</s>', '</s>', '<s>', '<s>', 'моё', 'второе', 'предложение', '.', '</s>', '</s>', '<s>', '<s>', 'еще', 'одно', 'предложение', '.', '</s>', '</s>']
все n-граммы
[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', 'моё'), ('<s>',), ('<s>', 'моё'), ('<s>', 'моё', 'первое'), ('моё',), ('моё', 'первое'), ('моё', 'первое', 'предложение'), ('первое',), ('первое', 'предложение'), ('первое', 'предложение', '!'), ('предложение',), ('предложение', '!'), ('предложение', '!', '</s>'), ('!',), ('!', '</s>'), ('!', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]
[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', 'моё'), ('<s>',), ('<s>', 'моё'), ('<s>', 'моё', 'второе'), ('моё',), ('моё', 'второе'), ('моё', 'второе', 'предложение'), ('второе',), ('второе', 'предложение'), ('второе', 'предложение', '.'), ('предложение',), ('предложение', '.'), ('предложение', '.', '</s>'), ('.',), ('.', '</s>'), ('.', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), (

5. Настроечное и тестовое множество (validation) предложений
   нужно обработать аналогично, добавить слова `<s>` и `<\s>`,
   вычислить n-граммы, причем в этот раз нас интересуют
   n-граммы при фиксированном $n$.

In [40]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams

sentence = ['мы', 'слова', 'одного', 'предложения']
padded_sentence = pad_both_ends(sentence, n)
print(list(padded_sentence))

# после распечатки padded_sentence стух
padded_sentence = pad_both_ends(sentence, n)
all_sentence_n_grams = ngrams(padded_sentence, n)
print(list(all_sentence_n_grams))

['<s>', '<s>', 'мы', 'слова', 'одного', 'предложения', '</s>', '</s>']
[('<s>', '<s>', 'мы'), ('<s>', 'мы', 'слова'), ('мы', 'слова', 'одного'), ('слова', 'одного', 'предложения'), ('одного', 'предложения', '</s>'), ('предложения', '</s>', '</s>')]


В принципе, рассмотренный выше `padded_everygram_pipeline` можно
было реализовать самостоятельно через функции `pad_both_ends` и
`ngrams`.

 6. Обучаем модель. Точнее, несколько моделей с разными видами
    сглаживания. Позже, мы будем их сравнивать.

In [ ]:
from nltk.lm import MLE, Lidstone, KneserNeyInterpolated

n_grams, words = padded_everygram_pipeline(n, sentences)
# создаём модель
model = MLE(n) # Модель MLE означает отсутствие сглаживания
# обучаем
model.fit(n_grams, words)

Кроме модели `MLE(n)` можно создать модель `Lidstone(gamma, n)`,
она означает модель, которую мы называли сглаживанием
Лапласса, и здесь `gamma` означает число, добавляемое в числитель.

Модель 
